In [ ]:
%matplotlib inline
import skmob
import pandas as pd
import skmob.measures.individual as ind_measure
import torch
import gpytorch
from gpytorch.kernels import RQKernel as RQ, RBFKernel as SE, \
PeriodicKernel as PER, ScaleKernel, LinearKernel as LIN, MaternKernel as MAT, \
SpectralMixtureKernel as SMK, PiecewisePolynomialKernel as PPK, CylindricalKernel as CYL
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from gpytorch.constraints import Interval
import time

# Import intra-package scripts
import utils.helper_func as helper_func
import utils.GP as GP
from utils.helper_func import dec_floor
import mobileDataToolkit.analysis as analysis
import mobileDataToolkit.preprocessing_v2 as preprocessing
import mobileDataToolkit.methods as methods
import mobileDataToolkit.metrics as metrics

import warnings
warnings.filterwarnings('ignore')

In [ ]:
file_path = "C:\\Users\\ekino\\OneDrive - UW\\GPR\\Data\\seattle_2000_all_obs_sampled.csv"
df = pd.read_csv(file_path, header=0)
df.columns

In [ ]:
# Add month column
df['month'] = pd.DatetimeIndex(df['datetime']).month

# Group by user ID, find month with third most observations (average)
df_m = df.groupby('UID').apply(lambda x: x[x['month'] == x['month'].value_counts().index[2]])

In [ ]:
# Keep only that month's data for each user
df_m = df_m.reset_index(drop=True)
df_m.shape

In [ ]:
df_m.UID.unique().shape # confirm that we have 50 users

In [ ]:
max_speed_kmh = 400 # for filtering out unrealistic speeds
spatial_radius_km = 0.3 # for compressing similar points using Douglas-Peucker algorithm

df_curr = df_m[df_m.UID == df_m.UID.unique()[1]]

tdf = skmob.TrajDataFrame(df_curr, latitude='orig_lat', longitude='orig_long', datetime='datetime')
f_tdf = skmob.preprocessing.filtering.filter(tdf, max_speed_kmh=max_speed_kmh, include_loops=False)
# Print the difference in number of rows
print("Number of rows before filtering: {}".format(tdf.shape[0]))
print("Number of rows after filtering: {}".format(f_tdf.shape[0]))
fc_tdf = skmob.preprocessing.compression.compress(f_tdf, spatial_radius_km=spatial_radius_km)
# Print the difference in number of rows
print("Number of rows after compression: {}".format(fc_tdf.shape[0]))
# Remove data points with uncertainty > 100m
fcu_tdf = fc_tdf[fc_tdf['orig_unc'] <= 100]
# Print the difference in number of rows
print("Number of rows after uncertainty filtering: {}".format(fcu_tdf.shape[0]))

df_curr = fcu_tdf

In [ ]:
# Calculate sci-kit mobility metrics
no_loc_gt = skmob.measures.individual._number_of_locations_individual(df_curr)
rg_gt = skmob.measures.individual._radius_of_gyration_individual(df_curr).squeeze()
k_rg_gt = skmob.measures.individual._k_radius_of_gyration_individual(df_curr).squeeze()
jumps_gt = skmob.measures.individual._jump_lengths_individual(df_curr).squeeze()
spat_burst_gt = helper_func.burstiness(jumps_gt)
loc_freq_gt = skmob.measures.individual._location_frequency_individual(df_curr, normalize=True) # matrix
rand_entr_gt = skmob.measures.individual._random_entropy_individual(df_curr).squeeze()
real_entr_gt = skmob.measures.individual._real_entropy_individual(df_curr).squeeze()
recency_gt = skmob.measures.individual._recency_rank_individual(df_curr).squeeze()  # matrix
freq_rank_gt = skmob.measures.individual._frequency_rank_individual(df_curr).squeeze() # matrix
uncorr_entr_gt = skmob.measures.individual._uncorrelated_entropy_individual(df_curr).squeeze()
max_dist_gt = skmob.measures.individual._maximum_distance_individual(df_curr).squeeze()
dist_straight_gt = skmob.measures.individual._distance_straight_line_individual(df_curr).squeeze()
waiting_time_gt = skmob.measures.individual._waiting_times_individual(df_curr).squeeze() # array
home_loc_gt = skmob.measures.individual._home_location_individual(df_curr) # tuple
max_dist_home_gt = skmob.measures.individual._max_distance_from_home_individual(df_curr).squeeze()
mob_network_gt = skmob.measures.individual._individual_mobility_network_individual(df_curr) # big matrix

In [ ]:
setattr(df_curr, f"no_loc_gp_pred", no_loc_gt)

In [ ]:
df_curr.no_loc_gp_pred

In [ ]:
bin_len_ls = [15, 20, 30, 60, 360, 1440, 10080] # Bin lengths to test: 15 min, 20 min, 30 min, 1 hr, 6 hr, 1 day, 1 week

upper_bound = dec_floor(analysis.tempOcp(df_curr, 'unix_min', bin_len=360))
curr_ocp = analysis.tempOcp(df_curr, 'unix_min', bin_len=360)
# See current temporal occupancy
print("Current temporal occupancy: {}".format(curr_ocp))
while True:
    try:
        # Choose random decimal between 0 and upper bound
        target_ocp = dec_floor(np.random.uniform(0.3, upper_bound))
        print("Target temporal occupancy: {}".format(target_ocp))
        # Simulate gaps in the user's data to match the target level
        gapped_user_data, train_index, new_ocp = analysis.simulate_gaps(df_curr, target_ocp, unix_col='unix_min', bin_len=360)
    except:
        continue
    break

In [ ]:
# Change name of 'lat' and 'lon' columns to 'orig_lat' and 'orig_long'
df_curr = df_curr.rename(columns={'lat': 'orig_lat', 'lng': 'orig_long'})

curr_mt = preprocessing.dp_MultiTrip(data=df_curr)
curr_mt.Multi_Trip_Preprocess(lat='orig_lat', long='orig_long', datetime='datetime')

In [ ]:
# Move 'unix_start_t' to before 'SaM'
cols = list(curr_mt.data.columns)
cols.insert(16, cols.pop(cols.index('unix_min')))
curr_mt.data = curr_mt.data.loc[:, cols]   
curr_mt.data.columns[-1]

In [ ]:
curr_mt.Multi_Trip_TrainTestSplit(test_start_date=None, test_end_date=None, 
                                training_index = set(gapped_user_data['unix_min']), lat='orig_lat', 
                                long='orig_long', datetime='datetime', unix='unix_min', inputstart='unix_min', inputend='day_6')

# See number of points in training and test sets
print("Number of points in training set: {}".format(len(curr_mt.X_train[:,0])))
print("Number of points in test set: {}".format(len(curr_mt.X_test[:,0])))

In [ ]:
%pwd

In [ ]:
# Visualize the training and test data in two subplots, one lat vs time and one long vs time
fig, ax = plt.subplots(2, 1, figsize=(10, 5), sharex=True)
ax[0].scatter(curr_mt.data[curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['unix_min'],
                curr_mt.data[curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['orig_lat'],
                color='blue', label='Training data', s=1)
ax[0].scatter(curr_mt.data[~curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['unix_min'],
                curr_mt.data[~curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['orig_lat'],
                color='red', label='Test data', s=1)
ax[0].set_ylabel('Latitude')
ax[1].scatter(curr_mt.data[curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['unix_min'],
                curr_mt.data[curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['orig_long'],
                color='blue', label='Training data', s=1)
ax[1].scatter(curr_mt.data[~curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['unix_min'],
                curr_mt.data[~curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['orig_long'],
                color='red', label='Test data', s=1)
ax[1].set_xlabel('Time (Unix)')
ax[1].set_ylabel('Longitude')
ax[1].legend()

# Save figure to file
fig.savefig('training_test_data.png', dpi=300)


In [ ]:
#scaler1 = MinMaxScaler(feature_range=(0, 100))
#scaler2 = MinMaxScaler(feature_range=(0, 10))
scaler3 = StandardScaler()

mean_lat = curr_mt.y_train[:,0].mean()
mean_long = curr_mt.y_train[:,1].mean()
std_lat = curr_mt.y_train[:,0].std()
std_long = curr_mt.y_train[:,1].std()

# Normalize the unix time such that it starts at 0
#tr_df.X_train[:,0] = tr_df.X_train[:,0] - tr_df.X_train[:,0].min()
#tr_df.X_test[:,0] = tr_df.X_test[:,0] - tr_df.X_train[:,0].min()

#unix_train = torch.tensor(np.float64(scaler1.fit_transform(curr_mt.X_train[:,0].reshape(-1,1))))
#secs_train = torch.tensor(scaler2.fit_transform(tr_df.X_train[:,1].reshape(-1,1))).float()
#unix_test = torch.tensor(np.float64(scaler1.transform(curr_mt.X_test[:,0].reshape(-1,1))))
#secs_test = torch.tensor(scaler2.transform(tr_df.X_test[:,1].reshape(-1,1))).float()

#X_train = torch.cat([unix_train, curr_mt.X_train[:, 1::]], -1)
#X_test = torch.cat([unix_test, curr_mt.X_test[:, 1::]], -1)

#X_train = tr_df.X_train.float()
#X_test = tr_df.X_test.float()

curr_mt.y_train = torch.tensor(np.float64(scaler3.fit_transform(curr_mt.y_train)))
curr_mt.y_test = torch.tensor(np.float64(scaler3.transform(curr_mt.y_test)))

n_dims = curr_mt.X_train.shape[1]


### Model Specification

In [ ]:
torch.set_default_tensor_type(torch.DoubleTensor)

In [ ]:
n_dims = curr_mt.X_train.shape[1]
print("Number of dimensions: {}".format(n_dims))

likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=2)

model = GP.MTGPRegressor(curr_mt.X_train, curr_mt.y_train, 
                         ScaleKernel(RQ(ard_num_dims=n_dims) * PER(active_dims=[0])) + ScaleKernel(RQ(ard_num_dims=n_dims) * PER(active_dims=[0])))

### Parameter Initialization

In [ ]:
# Set initial lengthscale guess as half the average length of gap in training set
init_lengthscale = curr_mt.data[curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['unix_min'].diff().mean() / 2

#scaled_unix_lengthscale = scaler1.transform(torch.tensor(init_lengthscale).reshape(-1,1)).item()

initializations = np.ones(n_dims - 1)
initializations = np.insert(initializations, 0, init_lengthscale)

model.covar_module.data_covar_module.kernels[0].base_kernel.kernels[0].lengthscale = initializations
model.covar_module.data_covar_module.kernels[1].base_kernel.kernels[0].lengthscale = initializations

# Set initial period lengths
model.covar_module.data_covar_module.kernels[0].base_kernel.kernels[1].period_length = 60*8
model.covar_module.data_covar_module.kernels[1].base_kernel.kernels[1].period_length = 60*24
#model.covar_module.data_covar_module.kernels[3].base_kernel.period_length = 60*12
#model.covar_module.data_covar_module.kernels[4].base_kernel.period_length = 60*6


In [ ]:
ls, mll = GP.training(model, curr_mt.X_train, curr_mt.y_train, lr=0.2)

In [ ]:
# Check model parameters (converting back to original scale)
print(model.covar_module.data_covar_module.kernels[0].base_kernel.lengthscale)
print(model.covar_module.data_covar_module.kernels[1].base_kernel.lengthscale)
print(model.covar_module.data_covar_module.kernels[0].outputscale)
print(model.covar_module.data_covar_module.kernels[1].outputscale)
print(model.likelihood.noise)

In [ ]:
# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model)

with torch.no_grad():
    log_ll = mll(model(curr_mt.X_train), curr_mt.y_train) * curr_mt.X_train.shape[0]
            
N = curr_mt.X_train.shape[0]
m = sum(p.numel() for p in model.hyperparameters())
bic = -2 * log_ll + m * np.log(N)

In [ ]:
predictions, mean = model.predict(curr_mt.X_test)

In [ ]:
# Write all model parameters to file
with open('model_params.txt', 'w') as f:
    f.write('Lengthscale: {}\n'.format(model.covar_module.data_covar_module.kernels[0].base_kernel.kernels[0].lengthscale))
    f.write('Period: {}\n'.format(model.covar_module.data_covar_module.kernels[0].base_kernel.period_length))
    f.write('Outputscale: {}\n'.format(model.covar_module.data_covar_module.kernels[0].outputscale))
    f.write('Noise: {}\n'.format(model.likelihood.noise))
    f.write('BIC: {}\n'.format(bic))


In [ ]:
# Visualize the predictions as we did earlier with training and testing points
fig, ax = plt.subplots(1, 2, figsize=(12, 6), sharex=True)
ax[0].scatter(curr_mt.data[curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['unix_min'],
                curr_mt.data[curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['norm_lat'],
                color='blue', label='Training data', s=1)
ax[0].scatter(curr_mt.data[~curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['unix_min'],  
                curr_mt.data[~curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['norm_lat'],
                color='green', label='Testing data', s=1)
ax[0].scatter(curr_mt.data[~curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['unix_min'],
                predictions.mean[:,0],
                color='red', label='Predictions', s=1)
ax[0].set_title('Latitude')
ax[0].set_ylabel('Latitude')

ax[1].scatter(curr_mt.data[curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['unix_min'],
                curr_mt.data[curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['norm_long'],
                color='blue', label='Training data', s=1)
ax[1].scatter(curr_mt.data[~curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['unix_min'],
                curr_mt.data[~curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['norm_long'],
                color='green', label='Testing data', s=1)
ax[1].scatter(curr_mt.data[~curr_mt.data['unix_min'].isin(set(gapped_user_data['unix_min']))]['unix_min'],
                predictions.mean[:,1],
                color='red', label='Predictions', s=1)
ax[1].set_title('Longitude')
ax[1].set_ylabel('Longitude')
ax[1].set_xlabel('Time (Unix)')
ax[1].legend()
plt.show()

In [ ]:
# Use smaller font
plt.rcParams.update({'font.size': 9})
# Make the font nicer
plt.rcParams.update({'font.family': 'serif'})
f, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.set_title('Predictions')
pd.DataFrame(mean.detach().numpy()).plot(x=1, y=0, kind='scatter',ax=ax, color='red', alpha=0.5, s=0.4, label='Predictions')
pd.DataFrame(curr_mt.y_test.detach().numpy()).plot(x=1, y=0, kind='scatter',ax=ax, color='blue', alpha=0.5, s=0.4, label='Actual')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.show()

In [ ]:
GP_res = metrics.average_eval(pd.Series(curr_mt.y_test[:,0]), pd.Series(curr_mt.y_test[:,1]), pd.Series(mean[:,0]), pd.Series(mean[:,1]))

In [ ]:
from statsmodels.tsa.holtwinters import Holt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
unix_min_tr = np.array(curr_mt.X_train[:,0]).astype(int)
unix_min_te = np.array(curr_mt.X_test[:,0]).astype(int)

In [ ]:
lat = pd.Series(curr_mt.y_train[:,0].tolist(), unix_min_tr)
lat_t = pd.Series(curr_mt.y_test[:,0].tolist(), unix_min_te)
# Replace duplicates (in time) with the mean of the two values
lat = lat.groupby(lat.index).mean().reset_index()
lat = pd.Series(lat[0].tolist(), lat['index'].tolist())
lat_tc = lat_t.groupby(lat_t.index).mean().reset_index()
lat_tc = pd.Series(lat_tc[0].tolist(), lat_tc['index'].tolist())
# Replace zeroes with positives close to zero
lat.replace(0, 0.000000001, inplace=True)


lon = pd.Series(curr_mt.y_train[:,1].tolist(), unix_min_tr)
lon_t = pd.Series(curr_mt.y_test[:,1].tolist(),unix_min_te)
# Replace duplicates (in time) with the mean of the two values
lon = lon.groupby(lon.index).mean().reset_index()
lon = pd.Series(lon[0].tolist(), lon['index'].tolist())
lon_tc = lon_t.groupby(lon_t.index).mean().reset_index()
lon_tc = pd.Series(lon_tc[0].tolist(), lon_tc['index'].tolist())
# Replace zeroes with positives close to zero
lon.replace(0, 0.000000001, inplace=True)

In [ ]:
smoothing_level = 0.1
ses_lat = SimpleExpSmoothing(lat, initialization_method="heuristic").fit(smoothing_level=smoothing_level, optimized=True)
pred_lat = ses_lat.predict(start=lat_tc.index[0], end=lat_tc.index[-1])
pred_lat_comp = pred_lat[pred_lat.index.isin(unix_min_te)]

ses_lon = SimpleExpSmoothing(lon, initialization_method="heuristic").fit(smoothing_level=smoothing_level, optimized=True)
pred_lon = ses_lon.predict(start=lon_tc.index[0], end=lon_tc.index[-1])
pred_lon_comp = pred_lon[pred_lon.index.isin(unix_min_te)]

In [ ]:
SES_res = metrics.average_eval(lat_tc, lon_tc, pred_lat_comp, pred_lon_comp)
SES_res

In [ ]:
GP_res

In [ ]:
LI_res = LI(curr_mt.X_train[:,0], curr_mt.X_test[:,0], curr_mt.y_train, curr_mt.y_test)

In [ ]:
# Compare LI_res and GP_res dictionaries
for key in LI_res.keys():
    print(key)
    print('GP: ', GP_res[key])
    print('LI: ', LI_res[key])
    print('SES: ', SES_res[key])
    print('')